In [1]:
import torch
import torch.nn as nn

In [2]:
text = open("input.txt").read()

text = text[:5000]  # training on only 5000 characters coz my gpu cant handle much :(

len(text)

5000

In [3]:
chars = list(set(text))

itos = {i:s for i, s in enumerate(chars)}

stoi = {s:i for i, s in itos.items()}

In [4]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [5]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        out, hidden = self.rnn(x, hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [6]:
input_size = len(chars)
hidden_size = 128
output_size = len(chars)
learning_rate = 0.01
num_epochs = 100

In [7]:
model = RNN(input_size, hidden_size, output_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
for epoch in range(num_epochs):
    hidden = model.init_hidden(batch_size=1)

    inputs = torch.tensor([stoi[ch] for ch in text[:-1]], dtype=torch.long, device=device)
    targets = torch.tensor([stoi[ch] for ch in text[1:]], dtype=torch.long, device=device)
    inputs_one_hot = torch.zeros(inputs.shape[0], len(chars), device=device)
    for i in range(inputs_one_hot.shape[0]):
        inputs_one_hot[i][inputs[i]] = 1
    inputs_one_hot = inputs_one_hot.unsqueeze(0)

    outputs, hidden = model(inputs_one_hot, hidden)
    loss = loss_fn(outputs.view(-1, output_size), targets.view(-1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 3.1185
Epoch [20/100], Loss: 2.7991
Epoch [30/100], Loss: 2.4264
Epoch [40/100], Loss: 2.2005
Epoch [50/100], Loss: 1.9668
Epoch [60/100], Loss: 1.7809
Epoch [70/100], Loss: 1.6249
Epoch [80/100], Loss: 1.3874
Epoch [90/100], Loss: 1.2221
Epoch [100/100], Loss: 1.0293


In [11]:
start_char = 'T'
num_chars = 500
hidden = model.init_hidden(batch_size=1)
input_char = start_char
output_text = start_char
with torch.no_grad():
    for i in range(num_chars):
        input_idx = torch.tensor([stoi[input_char]], dtype=torch.long, device=device)
        input_one_hot = torch.zeros(1, 1, input_size, device=device)
        input_one_hot.scatter_(2, input_idx.unsqueeze(1).unsqueeze(0), 1)

        output, hidden = model(input_one_hot, hidden)
        output_dist = output.squeeze().exp()

        top_char_idx = torch.multinomial(output_dist, 1)[0]
        output_char = itos[top_char_idx.item()]

        output_text += output_char

        input_char = output_char
    print(output_text)

The gheus?

Secong ur in
noursvindsire ath sitlle, beart,owe'. 
Aal: your bedledy, the bell: iu whal splok, wo
tieabe; the costhesito en tho ghay with or wealle ally at ie caremes ly
se cobell: sertistly pook.

Aly:
Heve, yeacchy gaicouss malt-coms ar musp forthis bow titllin;trfcounk, ally.

MENENIUS:
Eathom artice it burathin tareihe aik nitoze':
Ley uresty semetery god vermunto til  in citite s;rovedore beay ane thay soof lames,
Is arveros areas, mus arveueselvenoufmo toithe piti, the dakeeety
